## TASK

>Необходимо написать скрипт для парсинга диалогов из файла test_data.csv. 

Получившийся скрипт необходимо выложить в гит репозиторий и прислать ссылку в качестве результата прохождения тестового задания. Данные выкладывать в гит не следует. 

**Главные задачи, которые должен выполнять скрипт:**
- Извлекать реплики с приветствием – где менеджер поздоровался. 
- Извлекать реплики, где менеджер представил себя. 
- Извлекать имя менеджера. 
- Извлекать название компании. 
- Извлекать реплики, где менеджер попрощался.
- Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

**Рекомендации**:
- Сделать локальную копию файла test_data.csv, в исходнике никакие данные не менять!
- Можно создать дополнительное поле в таблице test_data.csv, куда будет сохраняться результат парсинга – например, напротив реплики в столбце “insight” можно ставить флаг того, что эта реплика с приветствием greeting=True.

In [ ]:
# !pip install -U spacy
!python -m spacy download ru_core_news_lg

In [20]:
import gc
import spacy
import pandas as pd
from pathlib import Path
PATH = Path('/content/drive/MyDrive/TZ/Bewise.ai')
assert spacy.__version__ >= '3.4.0', f'error version spacy update version loaded version is {spacy.__version__}'
spacy.__version__

'3.4.1'

In [ ]:
def skip_not_need(x: str) -> list:
    r"""
    we have drop no needs rows because:
       - grettings&names and org in first 3 row
       - parting ends 3 row
    """
    agg_val = x.tolist()
    return agg_val[:3] + agg_val[-3:]


def ner_ruler(name_model: str) -> spacy.lang:
    nlp = spacy.load(name_model)
    ruler = nlp.add_pipe('entity_ruler', config={"overwrite_ents": True})
    patterns = [
        {"label": "find_name", "pattern": [
            {'LEMMA': 'меня', "OP": "+"},
            {'LEMMA': 'звать', "OP": "?"},
            {'ENT_TYPE': 'PER', "OP": "+"}]},
        {'label': 'find_org', 'pattern': [
            {"LOWER": {"REGEX": r"(компан|фирм)"}, "OP": "+"},
            {"LOWER": {"REGEX": r"\w+"}}]},
        {'label': 'find_greetings', 'pattern': [
            {"LEMMA": {"IN": ['здравствовать', 'привет', 'добрый']}, "OP": "+"},
            {"LOWER": {"REGEX": r"(день|утро|вечер)", "OP": "?"}}]},
        {'label': 'find_goodbyes', 'pattern': [
            {"LEMMA": {"IN": ['до', 'всего', 'весь'], "OP": "+"}},
            {"LEMMA": {"IN": ['свидание', 'хороший', 'добрый'], "OP": "+"}}]}
    ]
    ruler.add_patterns(patterns)
    return nlp


def parse_same_ner(data: pd.DataFrame, model_name: str) -> tuple:
    r"""
    data: pd.DataFrame, dataframe where need find ner
    model_name: str, spacy pretrain model name
    return: tuple
    Explain:
        we add index to text
        add new columns
        aggregate text by manager, and take only six rows
        loads spacy model and add rule to ner
        loop data:
            split each text by '_' and get index position on data and text
            find ner and add to data by index
    """
    data['index_text'] = [str(k) + '_' + v for k, v in zip(range(len(data.text.values)), data.text.values)]
    new_col = ['find_name', 'find_org', 'find_goodbyes', 'find_greetings', 'greetings_goodbyes']
    list_text = data[data.role == 'manager'].groupby(['dlg_id'])['index_text'].apply(skip_not_need)
    model = ner_ruler(model_name)
    data[new_col] = [False] * len(new_col)
    tmp = []
    for i in range(len(list_text)):
        six_task = set()
        for doc in model.pipe(list_text[i]):
            idx, text = doc.text.split('_')
            idx = int(idx)
            doc = model(text)
            for ent in doc.ents:
                tmp.append(idx)              
                if ent.label_ == 'find_name':
                    data.loc[idx, new_col[0]] = ent.text.split()[-1]
                if ent.label_ == 'find_org':
                    data.loc[idx, new_col[1]] = ent.text
                if ent.label_ == 'find_goodbyes':
                    data.loc[idx, new_col[2]] = ent.text
                    six_task.add(1)
                if ent.label_ == 'find_greetings':
                    data.loc[idx, new_col[3]] = ent.text
                    six_task.add(2)
        if sum(six_task) == 3:
            data.loc[idx, new_col[-1]] = True
    return data.drop('index_text', axis = 1), tmp


ori = pd.read_csv(PATH / 'test_data.csv')
chk, tmp = parse_same_ner(ori, 'ru_core_news_lg')
gc.collect()

In [31]:
chk.loc[sorted(set(tmp))]

,dlg_id,line_n,role,text,find_name,find_org,find_goodbyes,find_greetings,greetings_goodbyes
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,ангелина,компания диджитал,False,False,False
108,0,108,manager,Всего хорошего до свидания,False,False,до свидания,False,False
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес з...,ангелина,компания диджитал,False,False,False
162,1,53,manager,Угу да вижу я эту почту хорошо тогда исправлю ...,False,False,всего хорошего,False,False
163,1,54,manager,До свидания,False,False,До свидания,False,False
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес з...,ангелина,компания диджитал,False,False,False
250,3,1,manager,Алло дмитрий добрый день,False,False,False,добрый день,False
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,максим,компания китобизнес,False,False,False
253,3,4,manager,Да дмитрий вот мне моя коллега анастасия подск...,False,False,False,False,False
300,3,51,manager,Угу все хорошо да понедельника тогда всего доб...,False,False,всего доброго,False,True
